In [8]:
from google.colab import files
uploaded = files.upload()  # Upload the Excel file manually

import pandas as pd
df = pd.read_excel("restaurant.xlsx")
df.head()


Saving restaurant.xlsx to restaurant.xlsx


,name,city,rating,rating_count,cuisine,link,address,item,order_date,sales_amount,Month,year,Age,Gender,Marital Status,Occupation
0,AB FOODS POINT,Abohar,--,Too Few Ratings,"Beverages,Pizzas",https://www.swiggy.com/restaurants/ab-foods-po...,"AB FOODS POINT, NEAR RISHI NARANG DENTAL CLINI...",Aloo Tikki Burger,2017-10-10,41241.0,October,2017,20.0,Female,Single,Student
1,Janta Sweet House,Abohar,4.4,50+ ratings,"Sweets,Bakery",https://www.swiggy.com/restaurants/janta-sweet...,"Janta Sweet House, Bazar No.9, Circullar Road,...",Veg Creamy Burger,2018-05-08,2100.0,May,2018,24.0,Female,Single,Student
2,theka coffee desi,Abohar,3.8,100+ ratings,Beverages,https://www.swiggy.com/restaurants/theka-coffe...,"theka coffee desi, sahtiya sadan road city",Cheese Burst Burger,2018-04-06,875.0,April,2018,22.0,Male,Single,Student
3,Singh Hut,Abohar,3.7,20+ ratings,"Fast Food,Indian",https://www.swiggy.com/restaurants/singh-hut-n...,"Singh Hut, CIRCULAR ROAD NEAR NEHRU PARK ABOHAR",Paneer Creamy Burger,2018-04-11,583.0,April,2018,22.0,Female,Single,Student
4,GRILL MASTERS,Abohar,--,Too Few Ratings,"Italian-American,Fast Food",https://www.swiggy.com/restaurants/grill-maste...,"GRILL MASTERS, ADA Heights, Abohar - Hanumanga...",Maxican Burger,2018-06-18,7176.0,June,2018,22.0,Male,Single,Student


In [10]:
import requests

# STEP 2: Basic cleaning

# Remove duplicates
df.drop_duplicates(inplace=True)

# Strip whitespace and standardize columns
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Convert data types
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
df['rating_count'] = df['rating_count'].astype(str).str.replace(',', '', regex=False)
df['rating_count'] = pd.to_numeric(df['rating_count'], errors='coerce')
df['sales_amount'] = df['sales_amount'].astype(str).replace('[₹$,]', '', regex=True)
df['sales_amount'] = pd.to_numeric(df['sales_amount'], errors='coerce')
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')
df['age'] = pd.to_numeric(df['age'], errors='coerce')
df['year'] = pd.to_numeric(df['year'], errors='coerce')

# Filter out unrealistic ages
df = df[(df['age'] >= 15) & (df['age'] <= 100)]

# STEP 3: Check for missing values
print("Missing values:")
print(df.isnull().sum())

# STEP 4: Check for data types
print("\nData types:")
print(df.dtypes)

# STEP 5: Check for invalid links (only structural)
df['link'] = df['link'].astype(str).str.strip()
df = df[df['link'].str.startswith(('http://', 'https://'), na=False)]

# Optional: Check if links work (sample only due to slowness)
def is_link_working(url):
    try:
        response = requests.head(url, timeout=5)
        return response.status_code < 400
    except:
        return False

# Apply on sample (e.g., first 50 links)
sample = df['link'].sample(min(50, len(df)), random_state=1)
link_check = sample.apply(is_link_working)
print(f"\nWorking links in sample: {link_check.sum()} / {len(link_check)}")

# STEP 6: Save cleaned version
df.to_excel(r"C:\Users\Kushal\Desktop\restaurant_cleaned.xlsx", index=False)

print("\n✅ Dataset cleaned and saved.")


Missing values:
name                  37
city                   0
rating             58100
rating_count      100000
cuisine               46
link                   0
address               39
item                   0
order_date             0
sales_amount           0
month                  0
year                   0
age                    0
gender                 0
marital_status         0
occupation             0
dtype: int64

Data types:
name                      object
city                      object
rating                   float64
rating_count             float64
cuisine                   object
link                      object
address                   object
item                      object
order_date        datetime64[ns]
sales_amount             float64
month                     object
year                       int64
age                      float64
gender                    object
marital_status            object
occupation                object
dtype: object


<ipython-input-10-c9127db14cf4>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['link'] = df['link'].astype(str).str.strip()



Working links in sample: 50 / 50

✅ Dataset cleaned and saved.


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 0 to 99999
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   name            99963 non-null   object        
 1   city            100000 non-null  object        
 2   rating          41900 non-null   float64       
 3   rating_count    0 non-null       float64       
 4   cuisine         99954 non-null   object        
 5   link            100000 non-null  object        
 6   address         99961 non-null   object        
 7   item            100000 non-null  object        
 8   order_date      100000 non-null  datetime64[ns]
 9   sales_amount    100000 non-null  float64       
 10  month           100000 non-null  object        
 11  year            100000 non-null  int64         
 12  age             100000 non-null  float64       
 13  gender          100000 non-null  object        
 14  marital_status  100000 non-null  object   

In [12]:
df.describe()

,rating,rating_count,order_date,sales_amount,year,age
count,41900.000000,0.0,100000,1.000000e+05,100000.000000,100000.000000
mean,3.905141,NaN,2019-01-10 19:37:41.375999744,5.189359e+03,2018.542370,24.628580
min,1.000000,NaN,2017-10-04 00:00:00,-1.000000e+00,2017.000000,18.000000
25%,3.700000,NaN,2018-05-14 00:00:00,1.710000e+02,2018.000000,23.000000
50%,4.000000,NaN,2018-12-24 00:00:00,5.190000e+02,2018.000000,24.000000
75%,4.200000,NaN,2019-09-03 00:00:00,2.847000e+03,2019.000000,26.000000
max,5.000000,NaN,2020-06-26 00:00:00,1.510944e+06,2020.000000,33.000000
std,0.460985,NaN,NaN,2.284853e+04,0.854513,2.971743


In [13]:
df.head(5)

,name,city,rating,rating_count,cuisine,link,address,item,order_date,sales_amount,month,year,age,gender,marital_status,occupation
0,AB FOODS POINT,Abohar,NaN,NaN,"Beverages,Pizzas",https://www.swiggy.com/restaurants/ab-foods-po...,"AB FOODS POINT, NEAR RISHI NARANG DENTAL CLINI...",Aloo Tikki Burger,2017-10-10,41241.0,October,2017,20.0,Female,Single,Student
1,Janta Sweet House,Abohar,4.4,NaN,"Sweets,Bakery",https://www.swiggy.com/restaurants/janta-sweet...,"Janta Sweet House, Bazar No.9, Circullar Road,...",Veg Creamy Burger,2018-05-08,2100.0,May,2018,24.0,Female,Single,Student
2,theka coffee desi,Abohar,3.8,NaN,Beverages,https://www.swiggy.com/restaurants/theka-coffe...,"theka coffee desi, sahtiya sadan road city",Cheese Burst Burger,2018-04-06,875.0,April,2018,22.0,Male,Single,Student
3,Singh Hut,Abohar,3.7,NaN,"Fast Food,Indian",https://www.swiggy.com/restaurants/singh-hut-n...,"Singh Hut, CIRCULAR ROAD NEAR NEHRU PARK ABOHAR",Paneer Creamy Burger,2018-04-11,583.0,April,2018,22.0,Female,Single,Student
4,GRILL MASTERS,Abohar,NaN,NaN,"Italian-American,Fast Food",https://www.swiggy.com/restaurants/grill-maste...,"GRILL MASTERS, ADA Heights, Abohar - Hanumanga...",Maxican Burger,2018-06-18,7176.0,June,2018,22.0,Male,Single,Student


In [14]:
print(df['rating'].unique())


[nan 4.4 3.8 3.7 3.6 4.  4.2 4.7 4.1 3.5 3.1 3.4 3.3 4.8 3.9 2.7 4.3 2.9
 4.5 2.5 3.2 2.4 4.6 3.  2.8 2.3 5.  2.6 2.2 1.4 1.9 4.9 2.1 1.3 2.  1.8
 1.6 1.1 1.5 1.  1.2 1.7]


In [15]:
print(df.isnull().sum())


name                  37
city                   0
rating             58100
rating_count      100000
cuisine               46
link                   0
address               39
item                   0
order_date             0
sales_amount           0
month                  0
year                   0
age                    0
gender                 0
marital_status         0
occupation             0
dtype: int64


In [16]:
print(df['rating'].unique()[:10])
print(df['rating_count'].unique()[:10])


[nan 4.4 3.8 3.7 3.6 4.  4.2 4.7 4.1 3.5]
[nan]


In [17]:
# Extract numbers from rating (e.g., "4.5/5" or "⭐ 4.2")
df['rating'] = df['rating'].astype(str).str.extract(r'([\d.]+)').astype(float)

# Clean and convert rating_count
df['rating_count'] = df['rating_count'].astype(str).str.replace(',', '', regex=False)
df['rating_count'] = df['rating_count'].str.extract(r'(\d+)').astype(float)


In [18]:
print(df[['rating', 'rating_count']].isnull().sum())


rating           58100
rating_count    100000
dtype: int64


In [19]:
# Option 1: Drop rows with any of these missing

# Option 2: Fill with placeholder if used in visualizations
df['name'].fillna("Unknown", inplace=True)
df['cuisine'].fillna("Not Specified", inplace=True)
df['address'].fillna("No Address", inplace=True)


<ipython-input-19-1ad32ee6c9c3>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['name'].fillna("Unknown", inplace=True)
<ipython-input-19-1ad32ee6c9c3>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.

In [20]:
df['rating'] = df['rating'].fillna(df['rating'].mean())


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 0 to 99999
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   name            100000 non-null  object        
 1   city            100000 non-null  object        
 2   rating          100000 non-null  float64       
 3   cuisine         100000 non-null  object        
 4   link            100000 non-null  object        
 5   address         100000 non-null  object        
 6   item            100000 non-null  object        
 7   order_date      100000 non-null  datetime64[ns]
 8   sales_amount    100000 non-null  float64       
 9   month           100000 non-null  object        
 10  year            100000 non-null  int64         
 11  age             100000 non-null  float64       
 12  gender          100000 non-null  object        
 13  marital_status  100000 non-null  object        
 14  occupation      100000 non-null  object   

In [22]:
# 1. Drop empty column
df.drop(columns=['rating_count'], inplace=True)

# 2. Standardize city, gender, marital_status
df['city'] = df['city'].str.title().str.strip()
df['gender'] = df['gender'].str.capitalize().str.strip()
df['marital_status'] = df['marital_status'].str.capitalize().str.strip()

# 3. Convert month (if needed)
try:
    df['month_num'] = pd.to_datetime(df['month'], format='%B').dt.month
except:
    df['month_num'] = pd.to_numeric(df['month'], errors='coerce')


In [25]:

# 1. Missing data report
print(df.isnull().sum())

# 2. Duplicate rows
print("Duplicates:", df.duplicated().sum())

# 3. Column data types
print(df.dtypes)

# 4. Unique values in key categorical fields
print(df['gender'].unique())
print(df['marital_status'].unique())


name              0
city              0
rating            0
cuisine           0
link              0
address           0
item              0
order_date        0
sales_amount      0
month             0
year              0
age               0
gender            0
marital_status    0
occupation        0
month_num         0
dtype: int64
Duplicates: 0
name                      object
city                      object
rating                   float64
cuisine                   object
link                      object
address                   object
item                      object
order_date        datetime64[ns]
sales_amount             float64
month                     object
year                       int64
age                      float64
gender                    object
marital_status            object
occupation                object
month_num                  int32
dtype: object
['Female' 'Male']
['Single' 'Married' 'Prefer not to say']


In [26]:
missing_score = df.isnull().mean().mean()  # % of missing values
duplicate_score = df.duplicated().mean()   # % of duplicate rows

cleanliness_score = 1 - (missing_score + duplicate_score)
print("Cleanliness Score (0–1):", round(cleanliness_score, 2))


Cleanliness Score (0–1): 1.0
